In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

23/02/24 14:06:54 WARN Utils: Your hostname, dangsg-mac.local resolves to a loopback address: 127.0.0.1; using 10.0.0.109 instead (on interface en0)
23/02/24 14:06:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/24 14:06:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark.version

'3.3.2'

!wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhvhv/fhvhv_tripdata_2021-06.csv.gz

In [4]:
from pyspark.sql import types

In [5]:
schema = types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True),
    types.StructField('Affiliated_base_number', types.StringType(), True)
])

In [6]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-06.csv.gz')

In [10]:
df.repartition(12).write.parquet("data/parquet/", mode="overwrite")

23/02/24 14:09:26 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [11]:
!ls -lh data/parquet/

total 591520
-rw-r--r--  1 dangsg  staff     0B Feb 24 14:09 _SUCCESS
-rw-r--r--  1 dangsg  staff    24M Feb 24 14:09 part-00000-caa48919-4414-4d3d-8607-755b13e131cc-c000.snappy.parquet
-rw-r--r--  1 dangsg  staff    24M Feb 24 14:09 part-00001-caa48919-4414-4d3d-8607-755b13e131cc-c000.snappy.parquet
-rw-r--r--  1 dangsg  staff    24M Feb 24 14:09 part-00002-caa48919-4414-4d3d-8607-755b13e131cc-c000.snappy.parquet
-rw-r--r--  1 dangsg  staff    24M Feb 24 14:09 part-00003-caa48919-4414-4d3d-8607-755b13e131cc-c000.snappy.parquet
-rw-r--r--  1 dangsg  staff    24M Feb 24 14:09 part-00004-caa48919-4414-4d3d-8607-755b13e131cc-c000.snappy.parquet
-rw-r--r--  1 dangsg  staff    24M Feb 24 14:09 part-00005-caa48919-4414-4d3d-8607-755b13e131cc-c000.snappy.parquet
-rw-r--r--  1 dangsg  staff    24M Feb 24 14:09 part-00006-caa48919-4414-4d3d-8607-755b13e131cc-c000.snappy.parquet
-rw-r--r--  1 dangsg  staff    24M Feb 24 14:09 part-00007-caa48919-4414-4d3d-8607-755b13e131cc-c000.snappy.parquet
-r

In [23]:
from pyspark.sql.functions import to_date, unix_timestamp, max, count

In [13]:
df.filter(to_date(df.pickup_datetime) == '2021-06-15').count()

452470

In [14]:
df = df.withColumn("duration", (unix_timestamp(df.dropoff_datetime) - unix_timestamp(df.pickup_datetime)) / 3600)

In [17]:
df.select(max(df.duration)).show()

+----------------+
|   max(duration)|
+----------------+
|66.8788888888889|
+----------------+



!wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/misc/taxi_zone_lookup.csv

In [18]:
df_zones = spark.read \
    .option("header", "true") \
    .csv("taxi_zone_lookup.csv")

In [31]:
df.registerTempTable('fhv_trip')

/opt/homebrew/Cellar/apache-spark/3.3.2/libexec/python/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [32]:
df_count_by_zone = spark.sql("""
    SELECT PULocationID, COUNT(*) as countByZone
    FROM fhv_trip
    GROUP BY PULocationID
    ORDER BY countByZone DESC
    LIMIT 1
""")

In [33]:
df_count_by_zone.show()

+------------+-----------+
|PULocationID|countByZone|
+------------+-----------+
|          61|     231279|
+------------+-----------+



In [35]:
df_count_by_zone.join(df_zones, df_count_by_zone.PULocationID == df_zones.LocationID, how="inner").show()

+------------+-----------+----------+--------+-------------------+------------+
|PULocationID|countByZone|LocationID| Borough|               Zone|service_zone|
+------------+-----------+----------+--------+-------------------+------------+
|          61|     231279|        61|Brooklyn|Crown Heights North|   Boro Zone|
+------------+-----------+----------+--------+-------------------+------------+

